In [1]:
import sys
import os
import pandas as pd
import numpy as np
import datetime as dt
import h5py
import json
import geojson
import requests
import base64
import sqlite3 as db
sys.path.append('../')

pd.options.display.max_rows=99

In [12]:
github_userName = 'Tanag3r'
ebird_token = 'j6c7l80ga2ib'
cnx = db.connect('trailheadDirectBirds_sous.db')
cur = cnx.cursor()

Obtain & clean trailhead data

In [3]:
##pull in trailhead stops from trailheadDirectBirds_sous db
trailheadRef = pd.DataFrame()
trailheadRef = pd.read_sql('select * from trailheadRef;',cnx)
trailheadRef.set_index('index',inplace=True)
trailheadRef.convert_dtypes()

,Route,StopName,Address,Latitude,Longitude
index,,,,,
0,IssaquahAlps,EastSunsetWay,"661-831 E Sunset Way, Issaquah, WA 98027",47.529635,-122.025119
1,IssaquahAlps,HighSchool,"Parking lot, The Rainier Trail, Issaquah, WA 9...",47.519345,-122.029801
2,IssaquahAlps,ChiricoTrail_PooPooPoint,"11400 Issaquah-Hobart Road Southeast, Issaquah...",47.499949,-122.02173
3,IssaquahAlps,SquakMountain,"13201 Squak Mountain Rd SE, Issaquah, WA 98027",47.481465,-122.053997
4,IssaquahAlps,NullMountain,<NA>,<NA>,<NA>
5,IssaquahAlps,MargaretsWay,"190th Ave SE, Issaquah, WA 98027",47.50662,-122.08666
6,MountSi,MountSi,,47.488966,-122.723044
7,MountSi,MountTeneriffe,"Mount Teneriffe Rd, North Bend, WA 98045",47.490104,-122.709182
8,MountSi,LittleSi,"SE Mt Si Rd, North Bend, WA 98045",47.489366,-122.753833


In [4]:
##For the sake of demonstration please assume the data in the 'trailheadRef' comes from a King County Metro service
##and/or is user-provided.

##set types for error checking, other cleaning
trailheadRef['Latitude'] = trailheadRef['Latitude'].fillna(0.00)
trailheadRef['Longitude'] = trailheadRef['Longitude'].fillna(0.00)
trailheadRef['Latitude'] = trailheadRef['Latitude'].astype('float64',errors='raise')
trailheadRef['Longitude'] = trailheadRef['Longitude'].astype('float64',errors='raise')
trailheadRef['CoordinateTest'] = ''
trailheadRef['CoordinateTest'] = trailheadRef['CoordinateTest'].astype('str')
trailheadRef['AddressTest'] = ''
trailheadRef['AddressTest'] = trailheadRef['AddressTest'].astype('str')
trailheadRef['failCode'] = ''
trailheadRef['failCode'] = trailheadRef['failCode'].astype('str')

##todo #10 finish this evaluator engine

##warnings:
trailheadRef.loc[(abs(trailheadRef['Latitude']) < 1) | (abs(trailheadRef['Longitude']) < 1), 'CoordinateTest'] = 'GEO10'
trailheadRef.loc[(trailheadRef['Address'].isna()) | (trailheadRef['Address'].isnull()) | (trailheadRef['Address'] == ''), 'AddressTest'] = 'GEO20'
##failures:
trailheadRef.loc[(trailheadRef['CoordinateTest'] != '') & (trailheadRef['AddressTest'] != ''), 'failCode'] = 'GEO30'
##update the test below when the coordinate fetch engine is complete to reflect failure of fetch
trailheadRef.loc[(trailheadRef['CoordinateTest'] != '') & (trailheadRef['AddressTest'] == ''), 'failCode'] = 'GEO31'

##TODO: #25 fetch coordinates of trailheads using Nominatim address service
##while trailheadRef['GeoTest'] is 'Fail':
    ##do the thing

##todo #9 write publisher for failed geoTest

trailheadRef_clean = pd.DataFrame()
trailheadRef_clean = trailheadRef[trailheadRef['failCode'] == '']
trailheadRef_clean.drop(columns=['CoordinateTest','AddressTest','failCode'],inplace=True,errors='ignore')


C:\Users\lukew\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Trailhead eBird hotspot update

1. Ask the eBird API for the latest list of hotspots for each trailhead
2. Add new eBird hotspots to the table 'Hotspots' in the trailheadDirectBirds_sous database
3. Update hotspot data

In [6]:
##get current hotspots from sous db
sous_trailheadHotspots = pd.read_sql('select * from hotspots',cnx)
sous_trailheadHotspots.set_index('index',inplace=True)
sous_trailheadHotspots.convert_dtypes()
##hard update on critical dtypes
sous_trailheadHotspots['latestObsDt'] = sous_trailheadHotspots['latestObsDt'].astype('datetime64[ns]')
sous_trailheadHotspots['latestUpdate'] = sous_trailheadHotspots['latestUpdate'].astype('datetime64[ns]')

In [7]:
##fetch all hotspots within 4 kilometers of each trailhead
import time
trailheadHotspots = []

for StopName in trailheadRef_clean.itertuples():
    time.sleep(0.3)
    url = '''https://api.ebird.org/v2/ref/hotspot/geo?lat={}&lng={}&dist=4&fmt=json'''.format(StopName.Latitude,StopName.Longitude)
    ebirdapi_auth_header = {'X-eBirdApiToken': ebird_token}
    ebird_request = requests.get(url,headers=ebirdapi_auth_header)
    ebird_response = pd.DataFrame(ebird_request.json())
    if ebird_request.status_code == requests.codes.ok:
        ebird_response['StopName'] = StopName.StopName
        trailheadHotspots.append(ebird_response)
    ebird_request.raise_for_status()

all_trailheadHotspots = pd.concat(trailheadHotspots)

In [8]:
##TODO #24 write script to pass in new hotspots and update existing hotspots --DONE
today = dt.datetime.today()
pd.DataFrame(all_trailheadHotspots)
all_trailheadHotspots['latestObsDt'] = all_trailheadHotspots['latestObsDt'].astype('datetime64[ns]')
all_trailheadHotspots['latestUpdate'] = today.date()

##sort, remove old data
frames = [all_trailheadHotspots,sous_trailheadHotspots]
hotspots = pd.concat(frames,ignore_index=True)
hotspots.sort_values(by=['StopName','locId','latestObsDt','latestUpdate'],ascending=False,inplace=True)
hotspots.drop_duplicates(subset=['StopName','locId'],keep='first',inplace=True)

In [13]:
##load to cooking
##this load into cooking triggers more scripts that update the production table 'Hotspots'
hotspots.to_sql(name='Hotspots_cooking',con=cnx,if_exists='replace')

Create a geoJSON polygon for each stop

In [10]:
##close the connection
cnx.close()